In [ ]:
# ReAct agent 核心组件

In [14]:
# SerpApi 库提供结构化的google搜索结果
# 安装库
# !pip install google-search-results



In [15]:
# 先实现工具的核心功能，然后构建一个通用的工具管理器

In [ ]:
# 搜索工具：接收一个查询字符串，然后返回搜索结果

from serpapi import SerpApiClient
import os

def search(query: str) -> str:
    """
    import os
    一个基于SerpApi的网页搜索引擎工具
    它会智能地解析搜索结果，优先返回直接答案或知识图谱信息
    """
    print(f"正在执行[SerpApi]网页搜索：{query}")
    try:
        api_key = os.getenv("SERPAPI_API_KEY")
        if not api_key:
            return "错误：SERPAPI_API_KEY未在 .env文件中配置"
        
        params = {
            "engine": "google",
            "q": query,
            "api_key": api_key,
            "gl": "cn", # 国家代码
            "hl": "zh-cn", #语言代码
        }

        client = SerpApiClient(params)
        results = client.get_dict()

        # 智能解析：优先寻找最直接的答案
        if "answer_box_list" in results:
            return "\n".join(results["answer_box_list"])
        if "answer_box" in results and "answer" in results["answer_box"]:
            return results["answer_box"]["answer"]
        if "knowledge_graph" in results and "description" in results["knowledge_graph"]:
            return results["knowledge_graph"]["description"]
        if "organic_results" in results and results["organic_results"]:
            # 如果没有直接答案，则返回前3个有机结果的摘要
            snippets = [
                f"[{i+1}] {res.get('title', '')}\n{res.get('snippet', '')}"
                for i, res in enumerate(results["organic_results"][:3])
            ]
            return "\n\n".join(snippets)
        
        return f"对不起，没有找到关于{query}的信息"
    
    except Exception as e:
        return f"搜索时发生错误：{e}"



In [17]:
# 构建通用的工具执行器
from typing import Dict, Any

class ToolExecutor:
    """
    一个工具执行器，负责管理和执行工具
    """
    def __init__(self):
        self.tools: Dict[str, Dict[str, Any]] = {}

    def registerTool(self, name: str, description: str, func: callable):
        """
        向工具箱中注册一个新工具
        """
        if name in self.tools:
            print(f"警告：工具'{name}'已存在，将被覆盖")
        self.tools[name] = {"description":description, "func": func}
        print(f"工具'{name}'已注册")

    def getTool(self, name: str) -> callable:
        """
        根据名称获取一个工具的执行函数
        """
        return self.tools.get(name, {}).get("func")
    
    def getAvailableTools(self) -> str:
        """
        获取所有可用工具的格式化描述字符串
        """
        return "\n".join([
            f"- {name}: {info['description']}"
            for name, info in self.tools.items()
        ])


In [ ]:
# --- 工具初始化与使用示例 ---
if __name__ == '__main__':
    import os
    from dotenv import load_dotenv

    # 加载.env文件环境变量
    load_dotenv()
    # 初始化工具执行器
    toolExecutor = ToolExecutor()

    # 注册搜索工具到工具执行器中
    search_description = "一个网页搜索引擎，当你需要回答关于时事、事实以及在你的知识库中找不到的信息时，应使用此工具。"
    toolExecutor.registerTool("Search", search_description, search)

    # 打印可用的工具
    print("\n--- 可用的工具 ---")
    print(toolExecutor.getAvailableTools())

    # 智能体的Action调用，这次问一个实时性的问题
    tool_name = "Search"
    tool_input = "闪光的哈萨维2内容梗概是什么？"
    print(f"\n--- 执行 Action: '{tool_name}':['{tool_input}'] ---")

    tool_function = toolExecutor.getTool(tool_name)
    if tool_function:
        observation = tool_function(tool_input)
        print("--- 观察(observation) ---")
        print(observation)
    else:
        print(f"错误：未找到名为'{tool_name}'的工具")

工具'Search'已注册

--- 可用的工具 ---
- Search: 一个网页搜索引擎，当你需要回答关于时事、事实以及在你的知识库中找不到的信息时，应使用此工具。

--- 执行 Action: 'Search':['闪光的哈萨维2内容梗概是什么？'] ---
正在执行[SerpApi]网页搜索：闪光的哈萨维2内容梗概是什么？
--- 观察(observation) ---
[1] 【钢科普·小说解说】《机动战士高达闪光的哈萨维》第二部的故事
【钢科普·小说解说】《机动战士高达闪光的哈萨维》第二部的故事. 是风风子哒 ... 背后竟然是全球航母的'潜规则. 03:00. 福建舰为什么不叠被子？背后竟然是全球航母的 ...

[2] 如何评价《闪光的哈萨维2：喀尔刻的魔女》？
简而言之，正是因为这种“心理学式”的描写，能否描绘出一个苦涩至极的青春故事——一个向往成为英雄、向往成为新人类的“凡俗之辈”，一个既未老成却也不再像少年 ...

[3] 机动战士高达闪光的哈萨维喀耳刻的魔女 - 维基百科
影片故事紧接上集发展，哈萨维·诺亚带领反地球联邦政府组织“马夫蒂”开始准备攻击地球联邦的阿德莱德会议。 联邦军上校凯奈斯·斯雷格则在准备策划阿德莱德会议的防卫作战时， ...
